# 准备数据集

nltk分词, jieba分词，构建词典，数字编码，形成batch

## 导入包

In [6]:
import os
import sys
import math

from collections import Counter
import numpy as np
import random 

import torch
import torch.nn as nn
import torch.nn.functional as F

import nltk
# 注意这里需要下载一个punkt包
# nltk.download()

import jieba

## 导入数据并分词

先导入数据：
- train.txt和dev.txt
- 每个文件多行，每一行都是英文句子，后面跟着中文翻译，两者用'\t'分开